In [3]:
import numpy as np
from matplotlib import pyplot as plt


## Plane boarding methods


In [105]:
class Plane:
    def __init__(self, n_rows=10, width=6, *, name="Test") -> None:
        n_rows += 1
        self.n_rows = n_rows
        self.width = width
        self.seats = np.zeros([n_rows, width])
        self.aisle = np.zeros([n_rows])

        self.name = name
        # print(self.seats)
        # print(self.aisle)
    def __repr__(self) -> str:
        return f"<Plane: {self.name}, {self.n_rows}, {self.width}>"

    # def __init__(self, model="B777") -> None:
    #     # võtab lennuki mudeli ja seeläbi konfi
    #     print("2")
    #     pass
def create_plane(model:str="A320")->Plane:
    if model == "A320":
        return Plane(32, 6, name=model)
    elif model == "lil":
        return Plane(10, 4, name="LiL")

create_plane()


<Plane: A320, 33, 6>

In [5]:
Plane(10, 32)


In [333]:
class Passenger:
    def __init__(self, seat="A1", overhead_bag=True) -> None:
        """Sõitja loomine

        :param seat: Koha kirjeldus nagu lennukis, täht+number (base 1), defaults to "A1"
        :param overhead_bag: kas hakkab mökutama, defaults to False
        :variable loc: ['out'], [aisle', x], ['seat', delay], ['done']
        """

        # seats are A,B,C,D,E,F,G... no interupts!

        # lisada is priority (lapsed jne)
        # lisada kiirus?

        self.seat = seat
        self.col, self.row = ord(seat[0])-65, int(seat[1:])

        self.overhead_bag = overhead_bag

        # ['out'] - pole lennukis
        # ['aisle', x] - mitmendal real ta koridoris on (TODO: lisada teine aisle võimalus)
        # ['seat', x] - mis tähe juures on,
        # ['done'] - selleks et aktiivseteks removeda
        self.loc = ['out']
        # self.goal = self.seat # selleks et kui peab välja ronima et mööda lasta
        # pigem mööda ronimine teha const kaaluga

        #print(self.col, self.row)
    def __repr__(self) -> str:
        return self.seat
    def get_info(self)->str:
        return f"<seat: {self.seat}, loc: {self.loc}, bag: {self.overhead_bag}>"


In [7]:
Passenger()


A1

In [8]:
plane = Plane()
p = Passenger()
plane.seats[p.row][p.col] = 1
plane.seats


array([[0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [91]:
def create_things() -> tuple[Plane, list[Passenger]]:
    n, w = 5, 2
    plane = Plane(n, w)

    char = [str(chr(a)) for a in range(65, 65+w)]
    num = [str(b) for b in range(1, n+1)]

    passengers = [Passenger(b+a) for a in num for b in char]
    return plane, passengers
def create_passengers_for_plane(plane:Plane, /, baggage_pct = 0.7)->list[Passenger]:
    w, n = plane.width, plane.n_rows
    char = [str(chr(a)) for a in range(65, 65+w)]
    num = [str(b) for b in range(1, n)]
    return [Passenger(b+a, overhead_bag=np.random.uniform(0,1) < baggage_pct) for a in num for b in char]

_ = create_passengers_for_plane(create_plane())

## Simple boarding
Every cycle is exactly one step, no need to care about the overhead luggage and other stuff like getting past people

In [40]:

def board_passengers(plane: Plane, passengers: list[Passenger]) -> int:
    """Võtab lennuki ning teatud järjestuses reisijad ning seejärel simuleerib peale minekut

    :param plane: lennuk, eeldatavasti tühi
    :param passengers: nimekiri sõitjatest et lennuk täita, järjekord oluline
    :return: kaua aega võtab
    """
    p_queue: list[Passenger] = list()  # aktiivselt töös passengerid
    print(p_queue)
    print(passengers)
    _ = Passenger()
    time_taken = 0

    #p.loc = ['aisle', 1]
    #plane.aisle[p.loc[1]] = 1
    p_queue.append(passengers.pop(0))

    while passengers or p_queue:

        if p_queue:
            p = p_queue.pop(0)
        
        print(f"töötleme {p.seat}")
        #print(f"{p.get_info()}")

        # kui saame lennukisse siseneda:
        if p.loc[0] == 'out' and plane.aisle[1] == 0:
            p.loc = ['aisle', 1]
            plane.aisle[p.loc[1]] = 1
            p_queue.append(p) 
            
        # kui oleme õiges reas astume vahekäiku
        elif p.loc[0]=='aisle' and p.loc[1] == p.row:
            # print("siin")
            plane.aisle[p.loc[1]] = 0
            plane.seats[p.row][p.col] = 1
            p.loc = ['done']
            print(f"{p} istume {p.row} {p.col}")
            # TODO: istmeni jõudmine sõltub sellest kas on vaja võtta mööda minemine või mitte

        # # oleme reas ja liigume istme poole
        # elif p.loc[0] == 'seat':
        #     if p.loc[1] == ord(p.seat)-65:
        #             p.loc = ['done']
        #             plane.seats[p.row][p.col] = 1
        #             print(f"{p.seat} istub")
        #     elif p.seat > p.loc[1]:
        #         p.loc[1] += 1
        #         p_queue.append(p)
        #     else:
        #         p.loc[1] -= 1
        #         p_queue.append(p)

        # general case: kui x+1 vaba, liigu sinna
        elif p.loc[0] == 'aisle' and plane.aisle[p.loc[1]+1] == 0:
            # liigume edasi
            plane.aisle[p.loc[1]] = 0
            plane.aisle[p.loc[1]+1] = 1
            p.loc[1] += 1
            p_queue.append(p)
            #print("liigume")
        else:
            p_queue.append(p)


            # võtame uue inimese arvestusse
        if plane.aisle[1] == 0 and len(passengers)>0:
            p_queue.append(passengers.pop(0))


        time_taken += 1

        #print(plane.aisle)
    print(plane.seats)
    return time_taken

a320 = create_plane()
passengers = create_passengers_for_plane(a320) 
board_passengers(a320, passengers)


[]
[A1, B1, C1, D1, E1, F1, A2, B2, C2, D2, E2, F2, A3, B3, C3, D3, E3, F3, A4, B4, C4, D4, E4, F4, A5, B5, C5, D5, E5, F5, A6, B6, C6, D6, E6, F6, A7, B7, C7, D7, E7, F7, A8, B8, C8, D8, E8, F8, A9, B9, C9, D9, E9, F9, A10, B10, C10, D10, E10, F10, A11, B11, C11, D11, E11, F11, A12, B12, C12, D12, E12, F12, A13, B13, C13, D13, E13, F13, A14, B14, C14, D14, E14, F14, A15, B15, C15, D15, E15, F15, A16, B16, C16, D16, E16, F16, A17, B17, C17, D17, E17, F17, A18, B18, C18, D18, E18, F18, A19, B19, C19, D19, E19, F19, A20, B20, C20, D20, E20, F20, A21, B21, C21, D21, E21, F21, A22, B22, C22, D22, E22, F22, A23, B23, C23, D23, E23, F23, A24, B24, C24, D24, E24, F24, A25, B25, C25, D25, E25, F25, A26, B26, C26, D26, E26, F26, A27, B27, C27, D27, E27, F27, A28, B28, C28, D28, E28, F28, A29, B29, C29, D29, E29, F29, A30, B30, C30, D30, E30, F30, A31, B31, C31, D31, E31, F31, A32, B32, C32, D32, E32, F32]
töötleme A1
töötleme A1
A1 istume 1 0
töötleme B1
töötleme B1
B1 istume 1 1
töötleme C1
tö

6335

## Advanced boarding
Passengers take time to get past people and take time to put things in the overhead locker

In [60]:
col = 2
seats = [1,0,0,   0,0,0]
middle = 3
delay = sum(seats[col:middle])
delay
seats[2:0]

[]

In [104]:

def board_passengers_2(plane: Plane, passengers: list[Passenger]) -> int:
    """Võtab lennuki ning teatud järjestuses reisijad ning seejärel simuleerib peale minekut

    :param plane: lennuk, eeldatavasti tühi
    :param passengers: nimekiri sõitjatest et lennuk täita, järjekord oluline
    :return: kaua aega võtab
    """
    p_queue: list[Passenger] = list()  # aktiivselt töös passengerid
    # print(p_queue)
    # print(passengers)
    _ = Passenger()
    time_taken = 0

    #p.loc = ['aisle', 1]
    #plane.aisle[p.loc[1]] = 1
    p_queue.append(passengers.pop(0))

    while passengers or p_queue:

        if p_queue:
            p = p_queue.pop(0)

        #print(f"töötleme {p.seat}")
        # print(f"{p.get_info()}")

        # kui saame lennukisse siseneda:
        if p.loc[0] == 'out' and plane.aisle[1] == 0:
            p.loc = ['aisle', 1]
            plane.aisle[p.loc[1]] = 1
            p_queue.append(p)

        # kui oleme õiges reas astume vahekäiku
        elif p.loc[0] == 'aisle' and p.loc[1] == p.row:
            # print("siin")
            delay = 0
            if p.overhead_bag:
                delay += 1

            middle = plane.width/2
            side = -1 if p.col < middle else 1  # -1 left side, 1 right side
            if side == -1:
                persons_in_da_way = sum(plane.seats[p.row][p.col:3])
            elif side == 1:
                persons_in_da_way = sum(plane.seats[p.row][3:p.col])

            delay += persons_in_da_way
            p.loc = ['seat', delay]

            #print(f"{p.seat} sai delay {delay}")
            p_queue.append(p)

        # general case: kui x+1 vaba, liigu sinna
        elif p.loc[0] == 'aisle' and plane.aisle[p.loc[1]+1] == 0:
            # liigume edasi
            plane.aisle[p.loc[1]] = 0
            plane.aisle[p.loc[1]+1] = 1
            p.loc[1] += 1
            p_queue.append(p)
            # print("liigume")

        elif p.loc[0] == 'seat':
            # lihtsalt vähendame delayd ühe võrra
            if p.loc[1] == 0:
                plane.aisle[p.row] = 0
                plane.seats[p.row][p.col] = 1
                #print(f"{p} istume {p.row} {p.col}")
                p.loc = ['done']
            else:
                p.loc[1] -= 1
                p_queue.append(p)

        else:
            p_queue.append(p)

            # võtame uue inimese arvestusse
        if plane.aisle[1] == 0 and len(passengers) > 0:
            p_queue.append(passengers.pop(0))

        time_taken += 1

        # print(plane.aisle)
    # print(plane.seats)
    return time_taken


a320 = create_plane()
passengers = create_passengers_for_plane(a320, baggage_pct=0.7)
#np.random.shuffle(passengers)
board_passengers_2(a320, passengers)


18142

## method to semi-shuffle passenger order
as people are trying to go to gate before group is called

In [170]:
from collections import Counter

In [217]:
arr = np.asarray(list(map(lambda x: int(x), np.random.lognormal(0,  0.95, 1000))))
# c = Counter(arr)
# plt.bar(c.keys(), c.values())

arr += 1
arr = 1/arr
np.sort(arr)

array([0.04166667, 0.05      , 0.05882353, 0.0625    , 0.07692308,
       0.07692308, 0.08333333, 0.08333333, 0.09090909, 0.09090909,
       0.09090909, 0.1       , 0.11111111, 0.11111111, 0.125     ,
       0.125     , 0.125     , 0.14285714, 0.14285714, 0.14285714,
       0.14285714, 0.14285714, 0.14285714, 0.14285714, 0.14285714,
       0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
       0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
       0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
       0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.2       ,
       0.2       , 0.2       , 0.2       , 0.2       , 0.2       ,
       0.2       , 0.2       , 0.2       , 0.2       , 0.2       ,
       0.2       , 0.2       , 0.2       , 0.2       , 0.2       ,
       0.2       , 0.2       , 0.2       , 0.2       , 0.2       ,
       0.2       , 0.2       , 0.2       , 0.2       , 0.2       ,
       0.2       , 0.2       , 0.2       , 0.2       , 0.2    

In [322]:
np.random.seed(0)
def shuffle_passengers(passengers:list[Passenger]) -> list[Passenger]:
    weights = np.asarray(list(np.random.lognormal(0,  0.95, len(passengers))))
    np.sort(weights)
    weights /= weights.sum()
    # print(sum(weights))
    # passenger_weights = zip(weights, passengers)
    return list(np.random.choice(passengers, len(passengers), replace=False, p=weights))
    
    # return passengers

plane = create_plane('lil')
passengers = create_passengers_for_plane(plane, baggage_pct=0.7)
passengers = shuffle_passengers(passengers)
print(passengers)

[D3, A7, A2, B7, B2, A6, C1, A9, A1, C7, C10, A3, D5, C2, B4, C9, D9, D4, D6, A4, C6, B8, B6, D2, B5, C8, D10, C5, B10, B9, D1, A10, C3, D7, B3, D8, A5, B1, A8, C4]


In [267]:
plane = create_plane('A320')
passengers = create_passengers_for_plane(plane, baggage_pct=0.7)
passengers = shuffle_passengers(passengers)
board_passengers_2(plane, passengers)

11609

## Board plane from back (perfect)

In [370]:
def order_back2front(passengers):
    sorted(passengers, key=lambda p: p.col, reverse=False)
    return passengers

np.random.seed(0)
plane = create_plane('A320')
passengers = create_passengers_for_plane(plane, baggage_pct=0.7)
passengers = order_back2front(passengers)
print(passengers)
print(f"Without randomness: {board_passengers_2(plane, passengers)}")

np.random.seed(0)
plane = create_plane('A320')
passengers = create_passengers_for_plane(plane, baggage_pct=0.7)
passengers = order_back2front(passengers)
passengers = shuffle_passengers(passengers)
print(f"With randomness: {board_passengers_2(plane, passengers)}")

[A1, B1, C1, D1, E1, F1, A2, B2, C2, D2, E2, F2, A3, B3, C3, D3, E3, F3, A4, B4, C4, D4, E4, F4, A5, B5, C5, D5, E5, F5, A6, B6, C6, D6, E6, F6, A7, B7, C7, D7, E7, F7, A8, B8, C8, D8, E8, F8, A9, B9, C9, D9, E9, F9, A10, B10, C10, D10, E10, F10, A11, B11, C11, D11, E11, F11, A12, B12, C12, D12, E12, F12, A13, B13, C13, D13, E13, F13, A14, B14, C14, D14, E14, F14, A15, B15, C15, D15, E15, F15, A16, B16, C16, D16, E16, F16, A17, B17, C17, D17, E17, F17, A18, B18, C18, D18, E18, F18, A19, B19, C19, D19, E19, F19, A20, B20, C20, D20, E20, F20, A21, B21, C21, D21, E21, F21, A22, B22, C22, D22, E22, F22, A23, B23, C23, D23, E23, F23, A24, B24, C24, D24, E24, F24, A25, B25, C25, D25, E25, F25, A26, B26, C26, D26, E26, F26, A27, B27, C27, D27, E27, F27, A28, B28, C28, D28, E28, F28, A29, B29, C29, D29, E29, F29, A30, B30, C30, D30, E30, F30, A31, B31, C31, D31, E31, F31, A32, B32, C32, D32, E32, F32]
Without randomness: 18078
With randomness: 11067


## Boarding plane totally randomly

In [275]:
np.random.seed(0)
plane = create_plane('A320')
passengers = create_passengers_for_plane(plane, baggage_pct=0.7)
np.random.shuffle(passengers)
board_passengers_2(plane, passengers)

11603

## Window - Middle - Aisle

In [298]:
import copy

In [363]:
def order_by_wma(passengers: list[Passenger]) -> list[Passenger]:
    windows = list(filter(lambda p: p.col==0 or p.col==5, passengers))
    middles = list(filter(lambda p: p.col==2 or p.col==4, passengers))
    aisles = list(filter(lambda p: p.col==1 or p.col==3, passengers))
    
    np.random.shuffle(windows)
    np.random.shuffle(middles)
    np.random.shuffle(aisles)
    return windows + middles + aisles


np.random.seed(0)
plane = create_plane('A320')
passengers = create_passengers_for_plane(plane, baggage_pct=0.7)
passengers = order_by_wma(passengers)
passengers2 = copy.deepcopy(passengers)
print(f"Without randomness: {board_passengers_2(plane, passengers)}")

np.random.seed(0)
plane = create_plane('A320')
passengers = create_passengers_for_plane(plane, baggage_pct=0.7)
passengers = order_by_wma(passengers)
passengers = shuffle_passengers(passengers)
print(f"With randomness: {board_passengers_2(plane, passengers)}")

Without randomness: 10662
With randomness: 10610


Without randomness: 10662
With randomness: 10610
